<a href="https://colab.research.google.com/github/martinpius/PYTORCH/blob/main/Simple_neural_net_using_pytorch.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [34]:
from google.colab import drive
drive.mount('/content/drive', force_remount = True)
try:
  COLAB = True
  import tensorflow as tf
  print(f"You are on CoLaB with tensorflow version: {tf.__version__}")
except Exception as e:
  print(f"{type(e)}: {e}\nplease load your drive....")
def time_fmt(t:float = 123.1819)->float:
  h = int(t / (60 * 60))
  m = int(t % (60 * 60) / 60)
  s = int(t % 60)
  return f"{h}: {m:>02}: {s:>05.2f}"
print(f">>>time testing\tplease wait...\n>>>time elapse:\t{time_fmt()}")


Mounted at /content/drive
You are on CoLaB with tensorflow version: 2.4.1
>>>time testing	please wait...
>>>time elapse:	0: 02: 03.00


In [35]:
#In this notebook we are going to build a simple neural network  using pytorch:
import torch
print(f"you are on CoLaB with pytorch version: {torch.__version__}")

#setup the gpu env:
if torch.cuda.is_available():
  device = torch.device('cuda')
else:
  device = torch.device('cpu')
import numpy as np
import time, datetime, os, sys
import tensorflow_datasets as tfds

you are on CoLaB with pytorch version: 1.8.1+cu101


In [36]:
import torch.nn as nn
from torch.utils.data import DataLoader
import torch.optim as optim
import torch.nn.functional as F
import torchvision.datasets as datasets
import torchvision.transforms as transforms

In [37]:
#We define a simple mlp using torch:

In [38]:
class MLP(nn.Module):
  def __init__(self, input_size, num_classes):
    super(MLP, self).__init__()
    self.fc1 = nn.Linear(input_size, 100)
    self.fc2 = nn.Linear(100, 50)
    self.out = nn.Linear(50, num_classes)
  
  def forward(self, x):
    x = F.relu(self.fc1(x))
    x = F.relu(self.fc2(x))
    x = self.out(x)
    return x
  

In [39]:
#Testing the class to see if it does what we want:
data = torch.rand(64, 784)
mlp = MLP(784, 10)
mlp(data).shape #Must return an output for 64 samples 

torch.Size([64, 10])

In [40]:
#Hyper parameters for our model:
batch_size, num_epochs, learning_rate = 64, 10, 0.001

#inputs-outputs dimensions
input_size = 784
output_size = 10


In [41]:
#Loading the MNIST dataset from torch_vission.datasets
train_dfm = datasets.MNIST(root = 'MNIST_torch/', train = True, transform = transforms.ToTensor(), download = True)
train_loader = DataLoader(dataset = train_dfm, batch_size = batch_size, shuffle = True)
test_dfm = datasets.MNIST(root = 'MNIST_torch', train = False, transform = transforms.ToTensor(), download = True)
test_loader = DataLoader(dataset = test_dfm, batch_size = batch_size, shuffle = True)

In [42]:
x_train_batch, y_train_batch = next(iter(train_loader))
print(f"{x_train_batch.shape}\n{y_train_batch.shape}")

torch.Size([64, 1, 28, 28])
torch.Size([64])


In [43]:
#Initialize the model , get the loss and optimizer
model = MLP(input_size = input_size, num_classes = output_size).to(device)

In [44]:
loss = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr = learning_rate)

In [45]:
#Training the model:
tic = time.time()
for epoch in range(num_epochs):
  print(f">>>training begin for epoch: {epoch}\tplease wait...")
  for batch_idx, (x_batch, target_batch) in enumerate(train_loader):
    #Get the data to cuda if available
    x_batch.to(device = device)
    target_batch.to(device = device)
    #reshaping the data to train a simple mlp
    x_batch = x_batch.reshape(x_batch.shape[0],-1) #flatten the image into 64, 784
    
    #fowrad pass:
    scores = model(x_batch)
    train_loss = loss(scores, target_batch)

    #backward pass:
    optimizer.zero_grad()
    train_loss.backward()

    #gradient descent (here we do adam_step)
    optimizer.step()

#now we can monitor the training and perform the validation computations:
def check_accuracy(loader, model):
  if loader.dataset.train:
    print('checking accuracy on training data')
  else:
    print('cheking accuracy on test data')
  num_correct = 0
  num_examples = 0
  model.eval()
  
  with torch.no_grad():#no need to compute the gradient again
    for x, y in loader:
      x = x.to(device = device)
      y = y.to(device = device)
      x = x.reshape(x.shape[0],-1)
      scores = model(x) #returns predictions of shape (batch, num_classes)
      _, predictions = scores.max(1) #Get the index for the maximum prob
      num_correct+= (predictions == y).sum()#how many are correctly classified out of a batch
      num_examples+=predictions.size(0)# 64 samples in a batch
    print(f"Got {num_correct}/{num_examples} with the accuracy of: {float(num_correct)/float(num_examples)*100:.2f}")
  model.train()

check_accuracy(train_loader, model)
check_accuracy(test_loader,model)
toc = time.time()
print(f"\ntrain and validation time for this model is: {time_fmt(toc - tic)}")

>>>training begin for epoch: 0	please wait...
>>>training begin for epoch: 1	please wait...
>>>training begin for epoch: 2	please wait...
>>>training begin for epoch: 3	please wait...
>>>training begin for epoch: 4	please wait...
>>>training begin for epoch: 5	please wait...
>>>training begin for epoch: 6	please wait...
>>>training begin for epoch: 7	please wait...
>>>training begin for epoch: 8	please wait...
>>>training begin for epoch: 9	please wait...
checking accuracy on training data
Got 59605/60000 with the accuracy of: 99.34
cheking accuracy on test data
Got 9743/10000 with the accuracy of: 97.43

train and validation time for this model is: 0: 01: 11.00
